# RPA Workshop 1 - 개인 별 수행

1. 2008.csv 파일을 데이터프레임으로 읽으세요.
- https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/HG7NV7

2. 해당 데이터는 미국 내 항공 기록입니다 (2008년)

3. Origin와 Dest는 각각 출발지 및 도착지 공항 코드입니다.

4. 위의 데이터프레임에 대해서 다음 내용을 수행하세요.

- 데이터프레임에 대한 결측치 조사

- Dest 공항 코드의 고유한 값별로 빈도 조사

- (선택) Origin와 Dest 값으로 Group By하여 도착 지연 시간의 평균 계산

- flight라는 폴더 만든 후 WD로 지정

- 전체 데이터프레임을 Dest 공항 코드별로 나눠서 엑셀 저장: 공항코드.xlsx 형태



In [ ]:
import os
import pandas as pd
os.chdir("/content")
df = pd.read_csv("2008.csv")
df.isnull().mean()

In [ ]:
df["Dest"].value_counts()

In [ ]:
df.groupby(['Origin'])["ArrDelay"].mean()
df.groupby(['Dest'])["ArrDelay"].mean()
# df.groupby(['Origin','Dest'])["ArrDelay"].mean()

In [29]:
if not os.path.exists('flight'):
    os.makedirs('flight')
os.chdir('flight')

In [ ]:
code = df["Dest"].value_counts().index
for i in code:
  data = df[ df.Dest == i]
  data.to_excel(i+".xlsx")

# for i in df["Dest"].unique():
#   df[ df.Dest == i].to_excel(i+".xlsx")

# RPA Workshop 2 - 같은 기관에서 오신 경우 / 관심사가 갖으신 경우 팀으로 수행하셔도 됩니다.

1. 공공데이터포털 또는 AI허브 등에서 관심을 갖는 분야에 대한 표 형태의 엑셀이나 CSV인 경우 행정 데이터를 찾으세요.
- 예: 00현황, 00민원 등에 대한 데이터
- 관심 분야를 찾기 어려우신 경우 첨부된 데이터 들을 확인해보세요.

2. 해당 데이터를 다운로드한 후, 데이터 프레임으로 읽어 보세요.

3. 해당 데이터와 관련된 주요 키워드를 도출한 후, 관련 뉴스를 수집하세요.

4. 수집된 뉴스에서 적정한 개수의 토픽을 발견하세요.

- ***1부터 4의 내용을 학습하신 코드를 이용해서 워드로 만들어서 다운로드 하세요***
- ***워크숍 종료 후 PDF를 공유하는 발표를 진행할 예정입니다***

In [34]:
!pip install konlpy
!pip install tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import os
import pandas as pd
from konlpy.tag import Okt, Kkma
from tqdm import tqdm
from bs4 import BeautifulSoup
from urllib import parse
import requests
import urllib.request
import numpy as np
import time
import csv

os.chdir("/content")
df = pd.read_csv('공정거래위원회_소비자 민원학습데이터 단순민원 처리용 학습데이터_20211227.csv', encoding='CP949')
data = df['해결명(SOLUTION_CRTR_NAME)']


#값을 저장할 비어있는 리스트를 만듬.
tokenized_data = []

# 불용어 정의
stopwords = ['을', '등', '것', '년', '수', '분기','위','해','월','일',
             '부터', '까지' '적', '의','가','이','은',
             '들','는','좀','잘','걍','과','도','를','으로',
             '자','에','와','한','하다', '점', '나',
             '호','만','명','닉','미']

# 형태소 분석기 OKT를 사용한 토큰화 작업 (다소 시간 소요)
okt = Kkma() #Kkma()

for sentence in tqdm(data): #리뷰 각각에 대해서 아래작업 반복
    tokenized_sentence = okt.nouns(sentence)  #각 리뷰에서 명사만 뽑아서 리스트
    #위에서 명사만 추출한 결과에서, 각 명사가 stopwords인지 체크하고
    #stopwords 아닌 명사만 사용함
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    tokenized_data.append(stopwords_removed_sentence)

len(tokenized_data)

from gensim import corpora
dictionary = corpora.Dictionary(tokenized_data) #사전 구성
corpus = [dictionary.doc2bow(text) for text in tokenized_data]

import gensim
NUM_TOPICS = 5
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS,
                                           id2word=dictionary, passes=15)

#각 토픽의 출현확률이 높은 단어를 지정된 갯수만큼 반환
topics = ldamodel.print_topics(num_words=20)

#위 내용을 출력
for topic in topics:
    print(topic)


<ipython-input-87-8b10082abc1e>:14: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('공정거래위원회_소비자 민원학습데이터 단순민원 처리용 학습데이터_20211227.csv', encoding='CP949')
  1%|          | 6868/605354 [23:55<54:52:05,  3.03it/s]

In [ ]:
#----------------------------------------------#
#변경할 부분
startdate = "2023.06.01"
enddate = "2023.06.23"
searchword ="불량품"#검색어
#----------------------------------------------#

base_url = "https://search.naver.com/search.naver?where=news&sm=tab_pge&query="+parse.urlencode([('',searchword)], doseq=True)+"&sort=1&photo=3&field=0&pd=3&ds="+startdate+"&de="+enddate+"&mynews=1&office_type=0&office_section_code=0&news_office_checked=&nso=so:dd,p:from"+startdate.replace(".", "")+"to"+enddate.replace(".", "")+",a:all&mynews=0&start="
titles = [ ]
urls = [ ]
i=1

while True:
 try:
    time.sleep(0.45)
    url = base_url+str(i)
    html = urllib.request.urlopen(url).read()
    soup = BeautifulSoup(html, 'html.parser')
    t_news = soup.select('a.news_tit')  #클래스 표시
    t_len = len(t_news)

    if t_len <1:
        break

    for j in np.arange(t_len)+1:
            try:
                titles.append(soup.select('a.news_tit')[j].text)
                urls.append(soup.select('a.info')[j]['href'])
            except Exception as e:
                continue
    i=i+10

 except Exception as e:
    continue

result = pd.DataFrame({"Url": urls, "Title":titles})

articles = []
dates = []

for i in result.Url:
   try:
    if( "news.naver.com" in i ): #바뀐곳
     print(i)
     time.sleep(0.45)
     html = urllib.request.urlopen(i).read()
     soup = BeautifulSoup(html, 'html.parser')

     news = soup.select('div#dic_area')[0].text #id #, class.
     news_date = soup.select('span.media_end_head_info_datestamp_time._ARTICLE_DATE_TIME')[0].text  #네이버 뉴스의 날짜 시간 부분 지정

     articles.append(news)
     dates.append(news_date.split()[0])

   except Exception as e:
    print(e)
    continue


# 수집된 내용을 표로 정리하기, news컬럼, date컬럼, likes컬럼
df = pd.DataFrame({'news':articles, 'date':dates})

print(df.isnull().values.any()) #null값이 있는지 체크
train_data = df.dropna(how = 'any') # Null 값이 존재하는 행 제거
print(train_data.isnull().values.any()) # Null 값이 존재하는지 확인

train_data['news'] = train_data['news'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")

#값을 저장할 비어있는 리스트를 만듬.
tokenized_data = []

# 불용어 정의
stopwords = ['을', '등', '것', '년', '수', '분기','위','해','월','일',
             '부터', '까지' '적', '의','가','이','은',
             '들','는','좀','잘','걍','과','도','를','으로',
             '자','에','와','한','하다', '점', '나',
             '호','만','명','닉','미']

# 형태소 분석기 OKT를 사용한 토큰화 작업 (다소 시간 소요)
okt = Kkma() #Kkma()

for sentence in tqdm(train_data['news']): #리뷰 각각에 대해서 아래작업 반복
    tokenized_sentence = okt.nouns(sentence)  #각 리뷰에서 명사만 뽑아서 리스트
    #위에서 명사만 추출한 결과에서, 각 명사가 stopwords인지 체크하고
    #stopwords 아닌 명사만 사용함
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    tokenized_data.append(stopwords_removed_sentence)

len(tokenized_data)

from gensim import corpora
dictionary = corpora.Dictionary(tokenized_data) #사전 구성
corpus = [dictionary.doc2bow(text) for text in tokenized_data]

import gensim
NUM_TOPICS = 5
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS,
                                           id2word=dictionary, passes=15)

#각 토픽의 출현확률이 높은 단어를 지정된 갯수만큼 반환
topics = ldamodel.print_topics(num_words=20)

#위 내용을 출력
for topic in topics:
    print(topic)

In [ ]:
!pip install python-docx

In [ ]:
from docx import Document

doc = Document()
#스타일 적용하기 (일반 텍스트, 글꼴은 ‘맑은고딕체’)
from docx.oxml.ns import qn
style = doc.styles['Normal']
style.font.name =  '맑은 고딕'
style._element.rPr.rFonts.set(qn('w:eastAsia'), '맑은 고딕')

#제목 넣기
doc.add_heading('워크샵 보고서, H1', level=1) #level: 0~6

#문단 넣기
doc.add_paragraph()
doc.paragraphs[0].text = "워크샵 2번 문제"

doc.add_paragraph()
doc.paragraphs[1].text = "1. 데이터셋이름:  공정거래위원회_소비자 민원학습데이터 단순민원 처리용 학습데이터_20211227"

doc.add_paragraph()
doc.paragraphs[2].text = "2. 데이터셋 컬럼 정보:  해결명(SOLUTION_CRTR_NAME)"

doc.add_paragraph()
doc.paragraphs[3].text = "3. 주요 키워드: 불량품"

doc.add_paragraph()
doc.paragraphs[4].text = "4. 수집된 뉴스 건수: 건"

doc.add_paragraph()
doc.paragraphs[5].text = "5. 토픽 개수: 5"

doc.add_paragraph()
doc.paragraphs[6].text = "6. 토픽 "

#이미지 추가
#doc.add_picture('이미지이름.jpg',width= Cm(16), height= Cm(9))

#표 추가
table = doc.add_table(rows = 5, cols = 2)
table.style = doc.styles['Table Grid']  #기본 스타일

#표 내용 채우기

#첫 행
row1 = table.rows[0].cells
# 첫 행의 각 열들에 접근해서 값 입력
row1[0].text = '토픽1'
row1[1].text = '토픽 해석'

#두번째 행
row2 = table.rows[1].cells
# 두번째 행의 각 열들에 접근해서 값 입력
row2[0].text = '토픽2'
row2[1].text = '토픽해석2'

#세번째 행
row3 = table.rows[2].cells
# 세번째 행의 각 열들에 접근해서 값 입력
row3[0].text = '토픽3....'
row3[1].text = '토픽해석3....'

#네번째 행
row4 = table.rows[2].cells
# 네번째 행의 각 열들에 접근해서 값 입력
row4[0].text = '토픽4....'
row4[1].text = '토픽해석4....'

#다섯번째 행
row5 = table.rows[2].cells
# 다섯번째 행의 각 열들에 접근해서 값 입력
row5[0].text = '토픽5....'
row5[1].text = '토픽해석5....'

#저장
doc.save('테스트.docx')